In [212]:
import pandas as pd
import re
from random import randrange
from PETRUS.g2p.g2p import G2PTranscriber
from PETRUS.stress.tonic import StressDetector

In [213]:
file_path = 'output/words.csv'
df = pd.read_csv(file_path)

In [214]:
def is_vowel(char):
    # Define a regex pattern for vowels including accented ones
    padrao = r'[aeiouáàâãäåéèêëíìîïóòôõöúùûüýÿ]'
    return bool(re.fullmatch(padrao, char, re.IGNORECASE))

def tem_acentos(palavra):
    # Regex para encontrar caracteres acentuados
    padrao = r'[áàâãäåéèêëíìîïóòôõöúùûüýÿ]'
    return bool(re.search(padrao, palavra))

def separate_syllables(word):
    syllables = G2PTranscriber(word, algorithm="ceci").get_syllables() 
    if 'i' in syllables:
        syllables = G2PTranscriber(word, algorithm="silva").get_syllables()
    return syllables

In [215]:
def find_matches(word):
    syllables = separate_syllables(word)
    syllables_count = len(syllables)
    stressed_syllable_position = StressDetector(word).get_stress_syllable(syllables)
    stressed_syllable_idx = len(syllables) - syllables.index(word[stressed_syllable_position[0]:stressed_syllable_position[1]])

    return df[
        (df['start_with_vowel'] == is_vowel(word[0])) & 
        (df['end_with_vowel'] == is_vowel(word[-1])) & 
        (df['stressed_syllable_idx'] == stressed_syllable_idx) &
        (df['syllable_count'] == syllables_count)
    ]

word = 'oxente'
matches = list(find_matches(word)['word'])
print(len(matches), matches, sep=': ')

140: ['abaixo', 'aberta', 'aberto', 'abismo', 'abrindo', 'abriu', 'acaba', 'acaso', 'aceita', 'aceito', 'acerca', 'achado', 'achava', 'acima', 'acordo', 'admira', 'afeto', 'agora', 'agosto', 'ainda', 'alberto', 'alcova', 'alegre', 'alfredo', 'alguma', 'almoço', 'altura', 'amada', 'amado', 'amante', 'amava', 'amiga', 'amigo', 'andando', 'andava', 'andrade', 'antiga', 'antigo', 'aquela', 'aquele', 'aquilo', 'ardente', 'artigo', 'artista', 'aspecto', 'assunto', 'augusta', 'augusto', 'aurora', 'eduardo', 'efeito', 'elisa', 'embaixo', 'embora', 'emprego', 'empresa', 'encontro', 'enfermo', 'engano', 'engenho', 'enorme', 'enquanto', 'enterro', 'entrada', 'entrando', 'entrava', 'entregue', 'ernesto', 'escada', 'escola', 'escolha', 'escolho', 'escrava', 'escravo', 'escreve', 'escrita', 'escrito', 'escuro', 'esforço', 'espada', 'espanto', 'espaço', 'espelho', 'espera', 'espero', 'esposa', 'esposo', 'esquerda', 'esquina', 'estado', 'estando', 'estava', 'esteja', 'estela', 'esteve', 'estilo', 'est

In [216]:
def generate_isorhythmic_verse(verse):
    new_verse = []
    for word in verse.split(" "):
        if len(word) > 1:
            matches = list(find_matches(word.lower())['word'])
            if len(matches) > 0:
                idx = randrange(len(matches))
                new_verse.append(matches[idx].title() if word.istitle() else matches[idx])
            else:
                new_verse.append(word)
        else:
            new_verse.append(word)

    return ' '.join(new_verse)

In [223]:
verse = 'Ouviram do ipiranga as margens plácidas'
generate_isorhythmic_verse(verse)

'Aplausos tu acontece ah cores séculos'